# **Setup**

In [ ]:
!pip install -qq langchain langchain-core langchain-google-genai langchain-community

In [2]:
import os

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [9]:
from google.colab import userdata
TAVILY_API_KEY = userdata.get('TRAVITY_API_KEY')

In [10]:
os.environ['GOOGLE_API_KEY'] = GEMINI_API_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3,google_api_key=GEMINI_API_KEY)

# **Define Tools**
Tools are just functions wrapped so the agent can call them.

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools import tool

# Example 1: Web search (requires TAVILY_API_KEY in env)
search_tool = TavilySearchResults(max_results=2)

# Example 2: Custom calculator tool
@tool
def multiply_numbers(a: int, b: int) -> str:
    """Multiply two integers and return the result as string."""
    return str(a * b)

tools = [search_tool, multiply_numbers]


# **Create the Agent**
Use LangChain’s AgentExecutor.

In [16]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use tools if needed."),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),

    '''
      The scratchpad is where the agent keeps notes:
      “User asked: …”
      “I need to search → use search_tool”
      “Got answer, now multiply”
      “Final answer: …”
      This is like the AI’s notebook for reasoning steps.
    '''
])


agent = create_tool_calling_agent(llm, tools, prompt)
'''
The agent is a wrapper:
It reads the prompt.
It sees what tools are available.
It decides: “Do I answer directly, or call a tool?”
It logs its thought process in the scratchpad.
'''


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# AgentExecutor = the runtime that manages reasoning loops.


In [17]:
response = agent_executor.invoke({
    "input": "Search who won the FIFA World Cup in 2018, then multiply the year by 2."
})
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'who won the FIFA World Cup in 2018'}`


[{'title': '2018 WORLD CUP FINAL: France 4-2 Croatia - YouTube', 'url': 'https://www.youtube.com/watch?v=GrsEAvRerTg', 'content': '# 2018 WORLD CUP FINAL: France 4-2 Croatia\n## FIFA\n23600000 subscribers\n544831 likes\n\n### Description\n67447733 views\nPosted: 15 Jul 2018\nWatch or stream for FREE the biggest World Cup archive of all time on FIFA+ \n👉  \n\nFrance won the 2018 FIFA World Cup Russia™ after defeating Croatia 4-2 in a thoroughly entertaining Final at the Luzhniki Stadium in Moscow in what was a fitting end to an outstanding tournament.', 'score': 0.9158166}, {'title': 'World Cup 2018: France beat Croatia 4-2 in World Cup final - BBC', 'url': 'https://www.bbc.com/sport/football/44754965', 'content': "BBC Sport\n\n# World Cup 2018: France beat Croatia 4-2 in World Cup final\n\n comments\n\n  1772 Comments\n\nShare page\n\nAbout sharing\n\